<a href="https://colab.research.google.com/github/snam1235/rnn_text_generation/blob/master/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import files
import pandas as pd
import io
import re
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import random
import time, math


uploaded = files.upload()
uploaded2 = files.upload()
uploaded3 = files.upload()
kanye = uploaded["kanye_verses.txt"].decode("utf-8").lower()
ts = uploaded2["cleaned_taylorLyrics.txt"].decode("utf-8").lower()
sp = uploaded3["Shakespeare.txt"].decode("utf-8").lower()


Saving kanye.txt to kanye (6).txt
11631


In [0]:

kanye = re.sub('[^a-z ]+',"",kanye)
ts =  re.sub('[^a-z ]+',"",ts)
sp =  re.sub('[^a-z ]+',"",sp)
# all letters used in the file, alphabets and whitespace
char_set = sorted(list(set(kanye)))



length = len(char_set)-1
for i in reversed(range(length)):
  if re.search('[a-z ,]',char_set[i]) == None:
   
    del char_set[i]


print(char_set)

char_len = len(char_set)
#dictionary mapping from char to int and int to char
char_to_int = dict((c, i) for i, c in enumerate(char_set))

int_to_char = dict((c, i) for i, c in enumerate(char_set))
unit_size = 10

def get_text(lyrics):
  split = lyrics.split('\n')
  split2 = [line for line in split if line.strip() != ""]
  text =''
  for line in split2:
    text = text+line
  return text


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers,dropout):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.isdropout = dropout
        if dropout == True:
           self.dropout = nn.Dropout(0.5)
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
      
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        if self.isdropout ==True:
          output = self.dropout(output)
        output = self.decoder(output.view(1, -1))
        
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))


In [ ]:
chunk_len = 100
sentences = []
next_chars = []
nexts =[]
inputs = []

def random_chunk(text):
    start_index = random.randint(0, len(text) - chunk_len)
    end_index = start_index + chunk_len + 1
    return text[start_index:end_index]



def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = char_to_int[string[c]]
    return Variable(tensor)


def random_training_set(text):    
    chunk = random_chunk(text)
    inp = char_tensor(chunk[:-1]).unsqueeze(1)
    target = char_tensor(chunk[1:]).unsqueeze(1)
   
    return inp, target
def evaluate(prime_str, predict_len, temperature,decoder):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = char_set[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


In [ ]:
def train(inp, target,decoder,criterion,decoder_optimizer):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
      
       
        
        loss += criterion(output, target[c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data / chunk_len


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
def init_model(char_len, hidden_size, n_layers,lr,n_ep,dropout,artist):
  decoder = RNN(char_len, hidden_size, char_len, n_layers,dropout)
  decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
  criterion = nn.CrossEntropyLoss()
  loss_avg = 0
  text= get_text(artist)
  for epoch in range(1, n_epochs + 1):
      
      loss = train(*random_training_set(text),decoder,criterion,decoder_optimizer)       
      loss_avg += loss
  
      if epoch % 100 == 0:
          print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
          print(evaluate('im', 100,0.8,decoder), '\n')
    
  plt.figure()
  plt.plot(all_losses)
     

In [ ]:
n_epochs = 2000
hidden_size = 100   
n_layers = 1
lr = 0.005
start = time.time()
all_losses = []

# 6th param is for adding a dropout layer, last param is the text type ts=taylor swift, kanye = kanye, sp =shakespeare
init_model(char_len, hidden_size, n_layers,lr,n_epochs,False,sp)